<a href="https://colab.research.google.com/github/KoreTane/interesting/blob/main/AnaliseAnalise_2024_12_15_20_20_59.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
from pyspark.sql import functions as F
import seaborn as sns
import pyspark.pandas as pd
import os
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler, StandardScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

files = {
    "order_reviews": "dbfs:/FileStore/tables/ecommerce/order_reviews.csv",
    "product_category_name_translation": "dbfs:/FileStore/tables/ecommerce/product_category_name_translation.csv",
    "orders": "dbfs:/FileStore/tables/ecommerce/orders.csv",
    "sellers": "dbfs:/FileStore/tables/ecommerce/sellers.csv",
    "products": "dbfs:/FileStore/tables/ecommerce/products.csv",
    "customers": "dbfs:/FileStore/tables/ecommerce/customers.csv",
    "order_items": "dbfs:/FileStore/tables/ecommerce/order_items.csv",
    "order_payments": "dbfs:/FileStore/tables/ecommerce/order_payments.csv",
    "geolocation": "dbfs:/FileStore/tables/ecommerce/geolocation.csv"
}

order_reviews = spark.read.csv(files["order_reviews"], inferSchema=True, header=True)
product_category_name_translation = spark.read.csv(files["product_category_name_translation"], inferSchema=True, header=True)
orders = spark.read.csv(files["orders"], inferSchema=True, header=True)
sellers = spark.read.csv(files["sellers"], inferSchema=True, header=True)
products = spark.read.csv(files["products"], inferSchema=True, header=True)
customers = spark.read.csv(files["customers"], inferSchema=True, header=True)
order_items = spark.read.csv(files["order_items"], inferSchema=True, header=True)
order_payments = spark.read.csv(files["order_payments"], inferSchema=True, header=True)
geolocation = spark.read.csv(files["geolocation"], inferSchema=True, header=True)

#display(order_reviews)
#display(product_category_name_translation)
#display(orders)
#display(sellers)
#display(products)
#display(customers)
#display(order_items)
#display(order_payments)
#display(geolocation)
# Definir o esquema do DataFrame para orders

In [ ]:
#display(order_items)

In [ ]:
order_reviews.printSchema()
product_category_name_translation.printSchema()
orders.printSchema()
sellers.printSchema()
products.printSchema()
customers.printSchema()
order_items.printSchema()
order_payments.printSchema()
geolocation.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)


In [ ]:
dataframes = {
    "Order Reviews": order_reviews,
    "Product Category Name Translation": product_category_name_translation,
    "Orders": orders,
    "Sellers": sellers,
    "Products": products,
    "Customers": customers,
    "Order Items": order_items,
    "Order Payments": order_payments,
    "Geolocation": geolocation
}

for name, df3 in dataframes.items():
    print(f"Descriptive Statistics for {name}:")
    df3.describe().show(truncate=False)

Descriptive Statistics for Order Reviews:
+-------+------------------+-----------------------------------------+---------------------+--------------------+----------------------+------------------------------------------------------------------+-----------------------------------------------+
|summary|review_id         |order_id                                 |review_score         |review_comment_title|review_comment_message|review_creation_date                                              |review_answer_timestamp                        |
+-------+------------------+-----------------------------------------+---------------------+--------------------+----------------------+------------------------------------------------------------------+-----------------------------------------------+
|count  |104161            |101926                                   |101782               |12005               |41083                 |95398                                                             

In [ ]:
# engenharia de recursos em orders

orders = orders.withColumn('order_purchase_timestamp', col('order_purchase_timestamp').cast('timestamp'))
orders = orders.withColumn('order_approved_at', col('order_approved_at').cast('timestamp'))
orders = orders.withColumn('order_delivered_carrier_date', col('order_delivered_carrier_date').cast('timestamp'))
orders = orders.withColumn('order_delivered_customer_date', col('order_delivered_customer_date').cast('timestamp'))
orders = orders.withColumn('order_estimated_delivery_date', col('order_estimated_delivery_date').cast('timestamp'))

# Calcular durações e criar novas colunas
orders = orders.withColumn('days_to_approval', datediff(col('order_approved_at'), col('order_purchase_timestamp')))
orders = orders.withColumn('days_to_carrier', datediff(col('order_delivered_carrier_date'), col('order_purchase_timestamp')))
orders = orders.withColumn('days_to_delivery', datediff(col('order_delivered_customer_date'), col('order_purchase_timestamp')))
orders = orders.withColumn('days_estimated_vs_actual_delivery', datediff(col('order_delivered_customer_date'), col('order_estimated_delivery_date')))

orders = orders.withColumn('order_approved_at', col('order_approved_at').cast('timestamp'))
order_items = order_items.withColumn('shipping_limit_date', col('shipping_limit_date').cast('timestamp'))

joined_df2 = order_items.join(orders, on='order_id')

# Calcular a diferença em dias entre shipping_limit_date e order_approved_at
joined_df2 = joined_df2.withColumn('days_to_shipping_limit', datediff(col('shipping_limit_date'), col('order_approved_at')))

order_items = joined_df2.select('order_id', 'order_item_id', 'product_id', 'seller_id',
                                        'shipping_limit_date', 'price', 'freight_value',
                                        'days_to_shipping_limit')


In [ ]:
products.describe()

Out[6]: DataFrame[summary: string, product_id: string, product_category_name: string, product_name_lenght: string, product_description_lenght: string, product_photos_qty: string, product_weight_g: string, product_length_cm: string, product_height_cm: string, product_width_cm: string]

In [ ]:
for table_name, df in dataframes.items():
    print(f"Análise da tabela: {table_name}")


    null_counts = {column: df.filter(col(column).isNull()).count() for column in df.columns}


    duplicate_count = df.count() - df.distinct().count()

Análise da tabela: Order Reviews
Análise da tabela: Product Category Name Translation
Análise da tabela: Orders
Análise da tabela: Sellers
Análise da tabela: Products
Análise da tabela: Customers
Análise da tabela: Order Items
Análise da tabela: Order Payments
Análise da tabela: Geolocation


In [ ]:
# Aqui podemos ver que o cliente insatisfeito, também faz questão de expressar insatisfação e que precisamos o dobro de clientes extremamente satisfeitos para preencher as lacunas das insatisfações. Também percebemos que o cliente bem atendido faz questão de expressar o seu contentamento, enquanto o consumidor neutro se mostra tambem menos engajado, o que faz total sentido.
result_df = order_reviews.alias("r").join(
    orders.alias("o"),
    on="order_id",
    how="inner"
).groupBy(
    "r.review_score"
).agg(
    F.count("r.review_id").alias("total_reviews"),
    F.count("r.review_comment_message").alias("total_comments"),
    F.count("r.review_comment_title").alias("total_comment_titles")
).orderBy(
    "r.review_score"
)

result_df.show()

+------------+-------------+--------------+--------------------+
|review_score|total_reviews|total_comments|total_comment_titles|
+------------+-------------+--------------+--------------------+
|           1|        11424|          8745|                1873|
|           2|         3151|          2145|                 478|
|           3|         8179|          3557|                 824|
|           4|        19142|          5976|                1735|
|           5|        57328|         20554|                6658|
+------------+-------------+--------------+--------------------+



In [ ]:
# crie estratégias de mkt e vendas para melhorar a eficiência do envio e reduzir cancelamentos, indisponibilidades e atrasos.

order_reviews = order_reviews.withColumn('days_to_response', datediff(col('review_answer_timestamp'), col('review_creation_date')))


reviews_by_status = order_reviews.join(orders, 'order_id').groupBy('order_status').agg(
    count('review_id').alias('review_count'),
    avg('review_score').alias('avg_review_score')
)

reviews_by_status.show()

+------------+------------+------------------+
|order_status|review_count|  avg_review_score|
+------------+------------+------------------+
|     shipped|        1043|  2.00862895493768|
|    canceled|         609|1.8111658456486042|
|    invoiced|         313|1.6613418530351438|
|     created|           3|2.3333333333333335|
|   delivered|       96361| 4.155716524320005|
| unavailable|         597|1.5309882747068677|
|  processing|         296| 1.277027027027027|
|    approved|           2|               2.5|
+------------+------------+------------------+



In [ ]:
# VENDEDOR
# Os vendedores se concentram em SP e apresentam uma média de avaliações moderada com alta variabilidade indicando inconsistência na satisfação dos clientes. Vendedores do estado de Goiás e RS, demonstram maior consistência e satisfação.

# caberia segmentar esses estados para uma analise aprofundada, nessa oportunidade não o faremos

order_reviews = order_reviews.withColumn('has_review_comment', when((col('review_comment_message').isNotNull()) | (col('review_comment_title').isNotNull()), 1).otherwise(0))

joined_reviews_sellers = order_reviews.join(order_items, 'order_id').join(sellers, 'seller_id')

seller_state_analysis = joined_reviews_sellers.groupBy('seller_state').agg(
    count('order_id').alias('order_count'),
    avg('review_score').alias('avg_review_score'),
    stddev('review_score').alias('stddev_review_score'),
    sum('has_review_comment').alias('has_comment_count')
).orderBy('avg_review_score', ascending=False)

seller_state_analysis.show()


+------------+-----------+------------------+-------------------+-----------------+
|seller_state|order_count|  avg_review_score|stddev_review_score|has_comment_count|
+------------+-----------+------------------+-------------------+-----------------+
|          PA|          8|               4.5| 0.9258200997725514|                1|
|          MS|         49| 4.469387755102041| 1.1565401134476943|               21|
|          RN|         56| 4.267857142857143| 1.3002247557958646|               19|
|          GO|        518| 4.254826254826255| 1.2504257954583504|              230|
|          RS|       2188| 4.214351005484461| 1.2701902795769873|              898|
|          MT|        145| 4.165517241379311| 1.1785925758166451|               66|
|          CE|         94| 4.138297872340425|  1.403536729543698|               47|
|          PE|        445| 4.132584269662921| 1.3230267714752741|              195|
|          MG|       8794| 4.105867637025244| 1.3306197749728046|           

In [ ]:
#CONSUMIDOR
# o consumidor de SP se mostra mais satisfeito possivelmente também por causa de uma logística eficiente. RS e Paraná também demonstram alta satisfação e consistência, enquanto o Rio de Janeiro pode estar enfrentando desafios que impactam a experiência do cliente ** novamente observamos que o cliente insatisfeito tende a se expressar de alguma forma
# também caberia uma investigação no RJ, entender po motivo da inquietude do cliente

orders_customers = orders.join(customers, 'customer_id')

joined_reviews_customers = orders_customers.join(order_reviews, 'order_id')

joined_reviews_customers = joined_reviews_customers.dropDuplicates(['order_id', 'customer_id', 'review_id'])

customer_state_analysis = joined_reviews_customers.groupBy('customer_state').agg(
    count('order_id').alias('order_count'),
    avg('review_score').alias('avg_review_score'),
    stddev('review_score').alias('stddev_review_score'),
    sum('has_review_comment').alias('review_comment_count')
).orderBy('order_count', ascending=False)

customer_state_analysis.show()

+--------------+-----------+------------------+-------------------+--------------------+
|customer_state|order_count|  avg_review_score|stddev_review_score|review_comment_count|
+--------------+-----------+------------------+-------------------+--------------------+
|            SP|      41690| 4.173950587670904|  1.281865634106384|               17312|
|            RJ|      12765|3.8749706227967096|  1.514694893377469|                5932|
|            MG|      11625|4.1361720430107525| 1.3059053645086525|                4920|
|            RS|       5483|4.1333211745394856| 1.3013297034087985|                2110|
|            PR|       5038| 4.180031758634379| 1.2694944972400062|                1977|
|            SC|       3623| 4.071763731714049| 1.3451080589151831|                1413|
|            BA|       3357|3.8608876973488235|  1.441141051801142|                1723|
|            DF|       2148| 4.064711359404097| 1.3543225698536907|                 876|
|            GO|     

In [ ]:
# clientes que pagam acima dos valores médios do mercado tendem a criar maiores expectativas e se não antedidas resultam em avaliações um pouco menores

order_price_avg = order_items.groupBy('order_id').agg(F.mean('price').alias('avg_price'))

overall_avg_price = order_price_avg.agg(F.mean('avg_price')).collect()[0][0]

order_price_avg = order_price_avg.withColumn(
    'Segmento Preço',
    F.when(order_price_avg['avg_price'] >= overall_avg_price * 1.80, '80% acima da média')
     .when(order_price_avg['avg_price'] <= overall_avg_price * 0.20, '80% abaixo da média')
     .otherwise('Na média')
)

order_price_reviews = order_price_avg.join(order_reviews, 'order_id').groupBy('Segmento Preço').agg(
    F.count('order_id').alias('order_count'),
    F.mean('review_score').alias('avg_review_score'),
    stddev('review_score').alias('stddev_review_score'),
    sum('has_review_comment').alias('review_comment_count')
)

order_price_reviews.show()

+-------------------+-----------+-----------------+-------------------+--------------------+
|     Segmento Preço|order_count| avg_review_score|stddev_review_score|review_comment_count|
+-------------------+-----------+-----------------+-------------------+--------------------+
|           Na média|      76473|4.110483438599244| 1.3250955818019767|               32767|
| 80% acima da média|      10784|4.061758160237389| 1.3929003839722456|                4964|
|80% abaixo da média|      11208| 4.10581727337616| 1.3156470510074691|                4445|
+-------------------+-----------+-----------------+-------------------+--------------------+



In [ ]:
# sem relação com o comprimento da descrição dos produtos testamos 10, 20 e 50
# Calcular a média de product_name_length
overall_avg_name_length = products.agg(F.mean('product_name_lenght')).collect()[0][0]


products_name_length = products.withColumn(
    'name_length_segment',
    F.when(products['product_name_lenght'] >= overall_avg_name_length * 1.20, '20% acima da média')
     .when(products['product_name_lenght'] <= overall_avg_name_length * 0.80, '20% abaixo da média')
     .otherwise('Na média')
)

name_length_analysis = products_name_length.join(order_items, 'product_id').join(order_reviews, 'order_id')

name_length_results = name_length_analysis.groupBy('name_length_segment').agg(
    F.countDistinct('order_id').alias('order_count'),
    F.mean('review_score').alias('avg_review_score'),
    F.count('review_comment_message').alias('review_comment_count'),
    F.count('review_comment_title').alias('review_comment_title_count')
)

name_length_results.show()


+-------------------+-----------+-----------------+--------------------+--------------------------+
|name_length_segment|order_count| avg_review_score|review_comment_count|review_comment_title_count|
+-------------------+-----------+-----------------+--------------------+--------------------------+
|           Na média|      66214|4.028456038135594|               32110|                      9064|
|20% abaixo da média|      16789|4.040808823529412|                7861|                      2132|
| 20% acima da média|      16083| 4.04059061306984|                7671|                      2238|
+-------------------+-----------+-----------------+--------------------+--------------------------+



In [ ]:
# sem relação com o volume do produto, testamos 20, 50, 80
# Calcular o volume do produto
products = products.withColumn('product_volume',
    products['product_length_cm'] * products['product_height_cm'] * products['product_width_cm']
)

overall_avg_volume = products.agg(F.mean('product_volume')).collect()[0][0]

products_volume = products.withColumn(
    'volume_segment',
    F.when(products['product_volume'] >= overall_avg_volume * 1.80, '80% acima da média')
     .when(products['product_volume'] <= overall_avg_volume * 0.20, '80% abaixo da média')
     .otherwise('Na média')
)

volume_analysis = products_volume.join(order_items, 'product_id').join(order_reviews, 'order_id')

volume_results = volume_analysis.groupBy('volume_segment').agg(
    F.countDistinct('order_id').alias('order_count'),
    F.mean('review_score').alias('avg_review_score'),
    F.count('review_comment_message').alias('review_comment_count'),
    F.count('review_comment_title').alias('review_comment_title_count')
)

volume_results.show()


+-------------------+-----------+-----------------+--------------------+--------------------------+
|     volume_segment|order_count| avg_review_score|review_comment_count|review_comment_title_count|
+-------------------+-----------+-----------------+--------------------+--------------------------+
|           Na média|      54552|4.040088802287138|               26520|                      7516|
| 80% acima da média|      14642|3.969311785000908|                7383|                      1677|
|80% abaixo da média|      29574|4.049518652226234|               13739|                      4241|
+-------------------+-----------+-----------------+--------------------+--------------------------+



In [ ]:
# comprar mais de um produto aumenta as chances de eventuais frustrações do cliente
# Calcular a média de order_item_id por order_id
order_items_avg_item = order_items.groupBy('order_id').agg(F.mean('order_item_id').alias('avg_order_item_id'))

overall_avg_item_id = order_items_avg_item.agg(F.mean('avg_order_item_id')).collect()[0][0]

order_items_avg_item = order_items_avg_item.withColumn(
    'item_id_segment',
    F.when(order_items_avg_item['avg_order_item_id'] >= overall_avg_item_id * 1.20, '20% acima da média')
     .when(order_items_avg_item['avg_order_item_id'] <= overall_avg_item_id * 0.80, '20% abaixo da média')
     .otherwise('Na média')
)

order_items_reviews_avg_item = order_items_avg_item.join(order_reviews, 'order_id').groupBy('item_id_segment').agg(
    F.count('order_id').alias('order_count'),
    F.mean('review_score').alias('avg_review_score'),
    F.count('review_comment_message').alias('review_comment_count'),
    F.count('review_comment_title').alias('review_comment_title_count')
)

order_items_reviews_avg_item.show()


+------------------+-----------+------------------+--------------------+--------------------------+
|   item_id_segment|order_count|  avg_review_score|review_comment_count|review_comment_title_count|
+------------------+-----------+------------------+--------------------+--------------------------+
|          Na média|      88699| 4.160001803853482|               35489|                     10119|
|20% acima da média|       9766|3.6015768994470614|                4964|                      1360|
+------------------+-----------+------------------+--------------------+--------------------------+



In [ ]:
# Separar shipping_limit_date em grupos de até 3 dias, até 5 dias e até 7 dias
order_items = order_items.withColumn(
    'shipping_group',
    F.when(F.datediff(order_items['shipping_limit_date'], order_items['shipping_limit_date'].cast('date')) <= 1, 'até 1 dias')
     .when(F.datediff(order_items['shipping_limit_date'], order_items['shipping_limit_date'].cast('date')) <= 2, 'até 2 dias')
     .when(F.datediff(order_items['shipping_limit_date'], order_items['shipping_limit_date'].cast('date')) <= 3, 'até 3 dias')
     .otherwise('mais de 7 dias')
)

# Join com reviews para calcular a média de review_score em cada grupo
shipping_reviews = order_items.join(order_reviews, 'order_id').groupBy('shipping_group').agg(
    F.count('order_id').alias('order_count'),
    F.mean('review_score').alias('avg_review_score'),
    F.count('review_comment_message').alias('review_comment_count'),
    F.count('review_comment_title').alias('review_comment_title_count')
)

shipping_reviews.show()


+--------------+-----------+-----------------+--------------------+--------------------------+
|shipping_group|order_count| avg_review_score|review_comment_count|review_comment_title_count|
+--------------+-----------+-----------------+--------------------+--------------------------+
|    até 1 dias|     112372|4.032472502046773|               47642|                     13434|
+--------------+-----------+-----------------+--------------------+--------------------------+



In [ ]:
# Existe relação da avaliação do consumidor com a esteira de envio dos pedidos, principalmente nos dias para envio
orders = orders.withColumn(
    'approval_group',
    F.when(orders['days_to_approval'] <= 3, 'até 3 dias')
     .when(orders['days_to_approval'] <= 5, '3 a 5 dias')
     .when(orders['days_to_approval'] <= 7, '5 a 7 dias')
     .otherwise('mais de 7 dias')
)

orders = orders.withColumn(
    'carrier_group',
    F.when(orders['days_to_carrier'] <= 3, 'até 3 dias')
     .when(orders['days_to_carrier'] <= 5, '3 a 5 dias')
     .when(orders['days_to_carrier'] <= 7, '5 a 7 dias')
     .otherwise('mais de 7 dias')
)

orders = orders.withColumn(
    'delivery_group',
    F.when(orders['days_to_delivery'] <= 3, 'até 3 dias')
     .when(orders['days_to_delivery'] <= 5, '3 a 5 dias')
     .when(orders['days_to_delivery'] <= 7, '5 a 7 dias')
     .otherwise('mais de 7 dias')
)

orders = orders.withColumn(
    'estimated_vs_actual_group',
    F.when(orders['days_estimated_vs_actual_delivery'] <= 3, 'até 3 dias')
     .when(orders['days_estimated_vs_actual_delivery'] <= 5, '3 a 5 dias')
     .when(orders['days_estimated_vs_actual_delivery'] <= 7, '5 a 7 dias')
     .otherwise('mais de 7 dias')
)

order_analysis = orders.join(order_reviews, 'order_id')

approval_reviews = order_analysis.groupBy('approval_group').agg(
    F.count('order_id').alias('order_count'),
    F.mean('review_score').alias('avg_review_score'),
    F.count('review_comment_message').alias('review_comment_count'),
    F.count('review_comment_title').alias('review_comment_title_count')
)

carrier_reviews = order_analysis.groupBy('carrier_group').agg(
    F.count('order_id').alias('order_count'),
    F.mean('review_score').alias('avg_review_score'),
    F.count('review_comment_message').alias('review_comment_count'),
    F.count('review_comment_title').alias('review_comment_title_count')
)

delivery_reviews = order_analysis.groupBy('delivery_group').agg(
    F.count('order_id').alias('order_count'),
    F.mean('review_score').alias('avg_review_score'),
    F.count('review_comment_message').alias('review_comment_count'),
    F.count('review_comment_title').alias('review_comment_title_count')
)

estimated_vs_actual_reviews = order_analysis.groupBy('estimated_vs_actual_group').agg(
    F.count('order_id').alias('order_count'),
    F.mean('review_score').alias('avg_review_score'),
    F.count('review_comment_message').alias('review_comment_count'),
    F.count('review_comment_title').alias('review_comment_title_count')
)

approval_reviews.show()
carrier_reviews.show()
delivery_reviews.show()
estimated_vs_actual_reviews.show()



+--------------+-----------+------------------+--------------------+--------------------------+
|approval_group|order_count|  avg_review_score|review_comment_count|review_comment_title_count|
+--------------+-----------+------------------+--------------------+--------------------------+
|    5 a 7 dias|         88|3.9545454545454546|                  38|                        12|
|    até 3 dias|      97153|4.0907125873622014|               40081|                     11298|
|    3 a 5 dias|       1757|4.0085372794536145|                 713|                       208|
|mais de 7 dias|        226| 2.898230088495575|                 145|                        50|
+--------------+-----------+------------------+--------------------+--------------------------+

+--------------+-----------+------------------+--------------------+--------------------------+
| carrier_group|order_count|  avg_review_score|review_comment_count|review_comment_title_count|
+--------------+-----------+-----------

In [ ]:
# O maior endividamento pode causar também maior insatisfação do cliente ou seja, também tem relação com o nosso target
def calculate_installments_group(order_payments, threshold):
    return order_payments.filter(order_payments['payment_installments'] > threshold).join(order_reviews, 'order_id').agg(
        F.count('order_id').alias('order_count'),
        F.mean('review_score').alias('avg_review_score')
    )

installment_groups = [3, 6, 9, 12, 15, 18, 21]
installment_results = {}

for threshold in installment_groups:
    group = calculate_installments_group(order_payments, threshold)
    installment_results[threshold] = group
    print(f'Perfil do Consumidor com > {threshold} Parcelas:')
    group.show()


Perfil do Consumidor com > 3 Parcelas:
+-----------+-----------------+
|order_count| avg_review_score|
+-----------+-----------------+
|      28429|4.034366316085687|
+-----------+-----------------+

Perfil do Consumidor com > 6 Parcelas:
+-----------+-----------------+
|order_count| avg_review_score|
+-----------+-----------------+
|      12190|3.996800656275636|
+-----------+-----------------+

Perfil do Consumidor com > 9 Parcelas:
+-----------+-----------------+
|order_count| avg_review_score|
+-----------+-----------------+
|       5663|3.957443051386191|
+-----------+-----------------+

Perfil do Consumidor com > 12 Parcelas:
+-----------+------------------+
|order_count|  avg_review_score|
+-----------+------------------+
|        185|3.8702702702702703|
+-----------+------------------+

Perfil do Consumidor com > 15 Parcelas:
+-----------+----------------+
|order_count|avg_review_score|
+-----------+----------------+
|         80|           3.675|
+-----------+----------------+

In [ ]:
# Definição da segmentação por Necessidade do Consumidor
segmentacoes_necessidade = { "beleza_saude": "Higiene Pessoal", "informatica_acessorios": "Tecnologia e Acessórios", "automotivo": "Automóveis e Ferramentas", "cama_mesa_banho": "Casa e Decoração", "moveis_decoracao": "Casa e Decoração", "esporte_lazer": "Esporte e Lazer", "perfumaria": "Higiene Pessoal", "utilidades_domesticas": "Casa e Decoração", "telefonia": "Tecnologia e Acessórios", "relogios_presentes": "Moda e Acessórios", "alimentos_bebidas": "Alimentação e Bebidas", "bebes": "Infantil", "papelaria": "Trabalho e Estudo", "tablets_impressao_imagem": "Tecnologia e Acessórios", "brinquedos": "Infantil", "telefonia_fixa": "Tecnologia e Acessórios", "ferramentas_jardim": "Automóveis e Ferramentas", "fashion_bolsas_e_acessorios": "Moda e Acessórios", "eletroportateis": "Casa e Decoração", "consoles_games": "Entretenimento", "audio": "Entretenimento", "fashion_calcados": "Moda e Acessórios", "cool_stuff": "Entretenimento", "malas_acessorios": "Moda e Acessórios", "climatizacao": "Casa e Decoração", "construcao_ferramentas_construcao": "Automóveis e Ferramentas", "moveis_cozinha_area_de_servico_jantar_e_jardim": "Casa e Decoração", "construcao_ferramentas_jardim": "Automóveis e Ferramentas", "fashion_roupa_masculina": "Moda e Acessórios", "pet_shop": "Animais de Estimação", "moveis_escritorio": "Casa e Decoração", "market_place": "Serviços e Outros", "eletronicos": "Tecnologia e Acessórios", "eletrodomesticos": "Casa e Decoração", "artigos_de_festas": "Entretenimento", "casa_conforto": "Casa e Decoração", "construcao_ferramentas_ferramentas": "Automóveis e Ferramentas", "agro_industria_e_comercio": "Automóveis e Ferramentas", "moveis_colchao_e_estofado": "Casa e Decoração", "livros_tecnicos": "Trabalho e Estudo", "casa_construcao": "Automóveis e Ferramentas", "instrumentos_musicais": "Entretenimento", "moveis_sala": "Casa e Decoração", "construcao_ferramentas_iluminacao": "Automóveis e Ferramentas", "industria_comercio_e_negocios": "Trabalho e Estudo", "alimentos": "Alimentação e Bebidas", "artes": "Entretenimento", "moveis_quarto": "Casa e Decoração", "livros_interesse_geral": "Entretenimento", "construcao_ferramentas_seguranca": "Automóveis e Ferramentas", "fashion_underwear_e_moda_praia": "Moda e Acessórios", "fashion_esporte": "Moda e Acessórios", "sinalizacao_e_seguranca": "Automóveis e Ferramentas", "pcs": "Tecnologia e Acessórios", "artigos_de_natal": "Entretenimento", "fashion_roupa_feminina": "Moda e Acessórios", "eletrodomesticos_2": "Casa e Decoração", "livros_importados": "Entretenimento", "bebidas": "Alimentação e Bebidas", "cine_foto": "Entretenimento", "la_cuisine": "Casa e Decoração", "musica": "Entretenimento", "casa_conforto_2": "Casa e Decoração", "portateis_casa_forno_e_cafe": "Casa e Decoração", "cds_dvds_musicais": "Entretenimento", "dvds_blu_ray": "Entretenimento", "flores": "Moda e Acessórios", "artes_e_artesanato": "Entretenimento", "fraldas_higiene": "Infantil", "fashion_roupa_infanto_juvenil": "Infantil", "seguros_e_servicos": "Serviços e Outros" }

# Criação de uma função UDF para mapear as segmentações
def segmentacoe_necessidade(categoria):
    return segmentacoes_necessidade.get(categoria, "Necessidades")

segmentacoe_necessidade_udf = udf(segmentacoe_necessidade, StringType())

# Adição da coluna de segmentação de sentimento ao DataFrame
products = products.withColumn(
    "segmentacao_necessidade",
    segmentacoe_necessidade_udf(col("product_category_name"))
)

In [ ]:
# Definição da segmentação de sentimento
segmentacoes = {
    "beleza_saude": "Satisfação Alta",
    "informatica_acessorios": "Misto",
    "automotivo": "Satisfação Alta",
    "cama_mesa_banho": "Neutro",
    "moveis_decoracao": "Satisfação Alta",
    "esporte_lazer": "Satisfação Alta",
    "perfumaria": "Satisfação Alta",
    "utilidades_domesticas": "Neutro",
    "telefonia": "Misto",
    "relogios_presentes": "Satisfação Alta",
    "alimentos_bebidas": "Satisfação Alta",
    "bebes": "Satisfação Alta",
    "papelaria": "Neutro",
    "tablets_impressao_imagem": "Misto",
    "brinquedos": "Satisfação Alta",
    "telefonia_fixa": "Neutro",
    "ferramentas_jardim": "Satisfação Alta",
    "fashion_bolsas_e_acessorios": "Satisfação Alta",
    "eletroportateis": "Neutro",
    "consoles_games": "Satisfação Alta",
    "audio": "Satisfação Alta",
    "fashion_calcados": "Misto",
    "cool_stuff": "Satisfação Alta",
    "malas_acessorios": "Satisfação Alta",
    "climatizacao": "Neutro",
    "construcao_ferramentas_construcao": "Satisfação Alta",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Satisfação Alta",
    "construcao_ferramentas_jardim": "Satisfação Alta",
    "fashion_roupa_masculina": "Misto",
    "pet_shop": "Satisfação Alta",
    "moveis_escritorio": "Satisfação Alta",
    "market_place": "Neutro",
    "eletronicos": "Misto",
    "eletrodomesticos": "Neutro",
    "artigos_de_festas": "Satisfação Alta",
    "casa_conforto": "Satisfação Alta",
    "construcao_ferramentas_ferramentas": "Satisfação Alta",
    "agro_industria_e_comercio": "Satisfação Alta",
    "moveis_colchao_e_estofado": "Satisfação Alta",
    "livros_tecnicos": "Misto",
    "casa_construcao": "Satisfação Alta",
    "instrumentos_musicais": "Satisfação Alta",
    "moveis_sala": "Satisfação Alta",
    "construcao_ferramentas_iluminacao": "Satisfação Alta",
    "industria_comercio_e_negocios": "Satisfação Alta",
    "alimentos": "Satisfação Alta",
    "artes": "Satisfação Alta",
    "moveis_quarto": "Satisfação Alta",
    "livros_interesse_geral": "Misto",
    "construcao_ferramentas_seguranca": "Satisfação Alta",
    "fashion_underwear_e_moda_praia": "Satisfação Alta",
    "fashion_esporte": "Satisfação Alta",
    "sinalizacao_e_seguranca": "Neutro",
    "pcs": "Neutro",
    "artigos_de_natal": "Satisfação Alta",
    "fashion_roupa_feminina": "Misto",
    "eletrodomesticos_2": "Neutro",
    "livros_importados": "Misto",
    "bebidas": "Satisfação Alta",
    "cine_foto": "Satisfação Alta",
    "la_cuisine": "Satisfação Alta",
    "musica": "Satisfação Alta",
    "casa_conforto_2": "Satisfação Alta",
    "portateis_casa_forno_e_cafe": "Satisfação Alta",
    "cds_dvds_musicais": "Satisfação Alta",
    "dvds_blu_ray": "Satisfação Alta",
    "flores": "Satisfação Alta",
    "artes_e_artesanato": "Satisfação Alta",
    "fraldas_higiene": "Satisfação Alta",
    "fashion_roupa_infanto_juvenil": "Satisfação Alta",
    "seguros_e_servicos": "Neutro"
}

# Criação de uma função UDF para mapear as segmentações
def segmentacao_sentimento(categoria):
    return segmentacoes.get(categoria, "Neutro")

segmentacao_udf = udf(segmentacao_sentimento, StringType())

# Adição da coluna de segmentação de sentimento ao DataFrame
products = products.withColumn(
    "segmentacao_sentimento",
    segmentacao_udf(col("product_category_name"))
)


In [ ]:
# Definição da segmentação por tipo
# Definição da segmentação por tipo
segmentacoes_tipo = { "beleza_saude": "Uso Pessoal",
                     "informatica_acessorios": "Reposição",
                     "automotivo": "Reposição",
                     "cama_mesa_banho": "Uso Pessoal",
                     "moveis_decoracao": "Decorativo",
                     "esporte_lazer": "Esportivo",
                     "perfumaria": "Uso Pessoal",
                     "utilidades_domesticas": "Reposição",
                     "telefonia": "Reposição",
                     "relogios_presentes": "Presente",
                     "alimentos_bebidas": "Reposição",
                     "bebes": "Uso Pessoal",
                     "papelaria": "Educacional",
                     "tablets_impressao_imagem": "Reposição",
                     "brinquedos": "Presente",
                     "telefonia_fixa": "Reposição",
                     "ferramentas_jardim": "Reposição",
                     "fashion_bolsas_e_acessorios": "Presente",
                     "eletroportateis": "Reposição",
                     "consoles_games": "Presente",
                     "audio": "Uso Pessoal",
                     "fashion_calcados": "Uso Pessoal",
                     "cool_stuff": "Presente",
                     "malas_acessorios": "Presente",
                     "climatizacao": "Reposição",
                     "construcao_ferramentas_construcao": "Reposição",
                     "moveis_cozinha_area_de_servico_jantar_e_jardim": "Decorativo",
                     "construcao_ferramentas_jardim": "Reposição",
                     "fashion_roupa_masculina": "Uso Pessoal",
                     "pet_shop": "Reposição",
                     "moveis_escritorio": "Reposição",
                     "market_place": "Uso Pessoal",
                     "eletronicos": "Reposição",
                     "eletrodomesticos": "Reposição",
                     "artigos_de_festas": "Presente",
                     "casa_conforto": "Decorativo",
                     "construcao_ferramentas_ferramentas": "Reposição",
                     "agro_industria_e_comercio": "Reposição",
                     "moveis_colchao_e_estofado": "Decorativo",
                     "livros_tecnicos": "Educacional",
                     "casa_construcao": "Reposição",
                     "instrumentos_musicais": "Uso Pessoal",
                     "moveis_sala": "Decorativo",
                     "construcao_ferramentas_iluminacao": "Reposição",
                     "industria_comercio_e_negocios": "Reposição",
                     "alimentos": "Reposição",
                     "artes": "Uso Pessoal",
                     "moveis_quarto": "Decorativo",
                     "livros_interesse_geral": "Educacional",
                     "construcao_ferramentas_seguranca": "Reposição",
                     "fashion_underwear_e_moda_praia": "Uso Pessoal",
                     "fashion_esporte": "Uso Pessoal",
                     "sinalizacao_e_seguranca": "Reposição",
                     "pcs": "Reposição",
                     "artigos_de_natal": "Presente",
                     "fashion_roupa_feminina": "Uso Pessoal",
                     "eletrodomesticos_2": "Reposição",
                     "livros_importados": "Educacional",
                     "bebidas": "Reposição",
                     "cine_foto": "Uso Pessoal",
                     "la_cuisine": "Decorativo",
                     "musica": "Uso Pessoal",
                     "casa_conforto_2": "Decorativo",
                     "portateis_casa_forno_e_cafe": "Reposição",
                     "cds_dvds_musicais": "Uso Pessoal",
                     "dvds_blu_ray": "Uso Pessoal",
                     "flores": "Presente",
                     "artes_e_artesanato": "Uso Pessoal",
                     "fraldas_higiene": "Reposição",
                     "fashion_roupa_infanto_juvenil": "Uso Pessoal",
                     "seguros_e_servicos": "Uso Pessoal" }

# Criação de uma função UDF para mapear as segmentações de tipo
def segmentacao_tipo(categoria):
    return segmentacoes_tipo.get(categoria, "Uso Pessoal")

segmentacao_tipo_udf = udf(segmentacao_tipo, StringType())

# Adição da coluna de segmentação de tipo ao DataFrame
products = products.withColumn(
    "segmentacao_tipo",
    segmentacao_tipo_udf(col("product_category_name"))
)

In [ ]:
# Definição da segmentação por validade
segmentacoes_validade = {
    "beleza_saude": "Média Duração",
    "informatica_acessorios": "Longa Duração",
    "automotivo": "Longa Duração",
    "cama_mesa_banho": "Média Duração",
    "moveis_decoracao": "Longa Duração",
    "esporte_lazer": "Média Duração",
    "perfumaria": "Longa Duração",
    "utilidades_domesticas": "Longa Duração",
    "telefonia": "Longa Duração",
    "relogios_presentes": "Longa Duração",
    "alimentos_bebidas": "Curta Duração",
    "bebes": "Curta Duração",
    "papelaria": "Média Duração",
    "tablets_impressao_imagem": "Longa Duração",
    "brinquedos": "Média Duração",
    "telefonia_fixa": "Longa Duração",
    "ferramentas_jardim": "Longa Duração",
    "fashion_bolsas_e_acessorios": "Longa Duração",
    "eletroportateis": "Média Duração",
    "consoles_games": "Longa Duração",
    "audio": "Longa Duração",
    "fashion_calcados": "Média Duração",
    "cool_stuff": "Longa Duração",
    "malas_acessorios": "Longa Duração",
    "climatizacao": "Longa Duração",
    "construcao_ferramentas_construcao": "Longa Duração",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Longa Duração",
    "construcao_ferramentas_jardim": "Longa Duração",
    "fashion_roupa_masculina": "Média Duração",
    "pet_shop": "Média Duração",
    "moveis_escritorio": "Longa Duração",
    "market_place": "Não Perecíveis",
    "eletronicos": "Longa Duração",
    "eletrodomesticos": "Longa Duração",
    "artigos_de_festas": "Média Duração",
    "casa_conforto": "Longa Duração",
    "construcao_ferramentas_ferramentas": "Longa Duração",
    "agro_industria_e_comercio": "Longa Duração",
    "moveis_colchao_e_estofado": "Longa Duração",
    "livros_tecnicos": "Não Perecíveis",
    "casa_construcao": "Longa Duração",
    "instrumentos_musicais": "Não Perecíveis",
    "moveis_sala": "Longa Duração",
    "construcao_ferramentas_iluminacao": "Longa Duração",
    "industria_comercio_e_negocios": "Longa Duração",
    "alimentos": "Curta Duração",
    "artes": "Não Perecíveis",
    "moveis_quarto": "Longa Duração",
    "livros_interesse_geral": "Não Perecíveis",
    "construcao_ferramentas_seguranca": "Longa Duração",
    "fashion_underwear_e_moda_praia": "Média Duração",
    "fashion_esporte": "Média Duração",
    "sinalizacao_e_seguranca": "Longa Duração",
    "pcs": "Longa Duração",
    "artigos_de_natal": "Média Duração",
    "fashion_roupa_feminina": "Média Duração",
    "eletrodomesticos_2": "Longa Duração",
    "livros_importados": "Não Perecíveis",
    "bebidas": "Curta Duração",
    "cine_foto": "Longa Duração",
    "la_cuisine": "Longa Duração",
    "musica": "Longa Duração",
    "casa_conforto_2": "Longa Duração",
    "portateis_casa_forno_e_cafe": "Longa Duração",
    "cds_dvds_musicais": "Longa Duração",
    "dvds_blu_ray": "Longa Duração",
    "flores": "Curta Duração",
    "artes_e_artesanato": "Não Perecíveis",
    "fraldas_higiene": "Curta Duração",
    "fashion_roupa_infanto_juvenil": "Média Duração",
    "seguros_e_servicos": "Não Perecíveis"
}

# Criação de uma função UDF para mapear as segmentações de validade
def segmentacao_validade(categoria):
    return segmentacoes_validade.get(categoria, "Não Perecíveis")

segmentacao_validade_udf = udf(segmentacao_validade, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacao_validade",
    segmentacao_validade_udf(col("product_category_name"))
)


In [ ]:
# Definição da segmentação por Utilização Principal
segmentacoes_utilizacao = {
    "beleza_saude": "Uso Diário",
    "informatica_acessorios": "Trabalho",
    "automotivo": "Trabalho",
    "cama_mesa_banho": "Uso Diário",
    "moveis_decoracao": "Lazer",
    "esporte_lazer": "Lazer",
    "perfumaria": "Uso Diário",
    "utilidades_domesticas": "Uso Diário",
    "telefonia": "Trabalho",
    "relogios_presentes": "Presente",
    "alimentos_bebidas": "Uso Diário",
    "bebes": "Uso Diário",
    "papelaria": "Trabalho",
    "tablets_impressao_imagem": "Trabalho",
    "brinquedos": "Lazer",
    "telefonia_fixa": "Trabalho",
    "ferramentas_jardim": "Trabalho",
    "fashion_bolsas_e_acessorios": "Uso Diário",
    "eletroportateis": "Uso Diário",
    "consoles_games": "Lazer",
    "audio": "Lazer",
    "fashion_calcados": "Uso Diário",
    "cool_stuff": "Lazer",
    "malas_acessorios": "Viagem",
    "climatizacao": "Uso Diário",
    "construcao_ferramentas_construcao": "Trabalho",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Uso Diário",
    "construcao_ferramentas_jardim": "Trabalho",
    "fashion_roupa_masculina": "Uso Diário",
    "pet_shop": "Uso Diário",
    "moveis_escritorio": "Trabalho",
    "market_place": "Trabalho",
    "eletronicos": "Trabalho",
    "eletrodomesticos": "Uso Diário",
    "artigos_de_festas": "Lazer",
    "casa_conforto": "Uso Diário",
    "construcao_ferramentas_ferramentas": "Trabalho",
    "agro_industria_e_comercio": "Trabalho",
    "moveis_colchao_e_estofado": "Uso Diário",
    "livros_tecnicos": "Educação",
    "casa_construcao": "Trabalho",
    "instrumentos_musicais": "Lazer",
    "moveis_sala": "Uso Diário",
    "construcao_ferramentas_iluminacao": "Trabalho",
    "industria_comercio_e_negocios": "Trabalho",
    "alimentos": "Uso Diário",
    "artes": "Lazer",
    "moveis_quarto": "Uso Diário",
    "livros_interesse_geral": "Educação",
    "construcao_ferramentas_seguranca": "Trabalho",
    "fashion_underwear_e_moda_praia": "Uso Diário",
    "fashion_esporte": "Lazer",
    "sinalizacao_e_seguranca": "Trabalho",
    "pcs": "Trabalho",
    "artigos_de_natal": "Lazer",
    "fashion_roupa_feminina": "Uso Diário",
    "eletrodomesticos_2": "Uso Diário",
    "livros_importados": "Educação",
    "bebidas": "Uso Diário",
    "cine_foto": "Lazer",
    "la_cuisine": "Uso Diário",
    "musica": "Lazer",
    "casa_conforto_2": "Uso Diário",
    "portateis_casa_forno_e_cafe": "Uso Diário",
    "cds_dvds_musicais": "Lazer",
    "dvds_blu_ray": "Lazer",
    "flores": "Presente",
    "artes_e_artesanato": "Lazer",
    "fraldas_higiene": "Uso Diário",
    "fashion_roupa_infanto_juvenil": "Uso Diário",
    "seguros_e_servicos": "Trabalho"
}

# Criação de uma função UDF para mapear as segmentações de utilização principal
def segmentacao_utilizacao(categoria):
    return segmentacoes_utilizacao.get(categoria, "Uso Diário")

segmentacao_utilizacao_udf = udf(segmentacao_utilizacao, StringType())

# Adição da coluna de segmentação de utilização principal ao DataFrame
products = products.withColumn(
    "segmentacao_utilizacao",
    segmentacao_utilizacao_udf(col("product_category_name"))
)


In [ ]:
# Definição da segmentação por Funcionalidade Principal
segmentacoes_funcionalidade = {
    "beleza_saude": "Conforto",
    "informatica_acessorios": "Trabalho",
    "automotivo": "Especializado",
    "cama_mesa_banho": "Essencial",
    "moveis_decoracao": "Conforto",
    "esporte_lazer": "Entretenimento",
    "perfumaria": "Conforto",
    "utilidades_domesticas": "Essencial",
    "telefonia": "Trabalho",
    "relogios_presentes": "Conforto",
    "alimentos_bebidas": "Essencial",
    "bebes": "Essencial",
    "papelaria": "Trabalho",
    "tablets_impressao_imagem": "Trabalho",
    "brinquedos": "Entretenimento",
    "telefonia_fixa": "Trabalho",
    "ferramentas_jardim": "Especializado",
    "fashion_bolsas_e_acessorios": "Conforto",
    "eletroportateis": "Essencial",
    "consoles_games": "Entretenimento",
    "audio": "Entretenimento",
    "fashion_calcados": "Conforto",
    "cool_stuff": "Entretenimento",
    "malas_acessorios": "Conforto",
    "climatizacao": "Essencial",
    "construcao_ferramentas_construcao": "Especializado",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Essencial",
    "construcao_ferramentas_jardim": "Especializado",
    "fashion_roupa_masculina": "Conforto",
    "pet_shop": "Essencial",
    "moveis_escritorio": "Trabalho",
    "market_place": "Trabalho",
    "eletronicos": "Trabalho",
    "eletrodomesticos": "Essencial",
    "artigos_de_festas": "Entretenimento",
    "casa_conforto": "Conforto",
    "construcao_ferramentas_ferramentas": "Especializado",
    "agro_industria_e_comercio": "Especializado",
    "moveis_colchao_e_estofado": "Conforto",
    "livros_tecnicos": "Trabalho",
    "casa_construcao": "Especializado",
    "instrumentos_musicais": "Entretenimento",
    "moveis_sala": "Conforto",
    "construcao_ferramentas_iluminacao": "Especializado",
    "industria_comercio_e_negocios": "Especializado",
    "alimentos": "Essencial",
    "artes": "Entretenimento",
    "moveis_quarto": "Conforto",
    "livros_interesse_geral": "Entretenimento",
    "construcao_ferramentas_seguranca": "Especializado",
    "fashion_underwear_e_moda_praia": "Conforto",
    "fashion_esporte": "Conforto",
    "sinalizacao_e_seguranca": "Especializado",
    "pcs": "Trabalho",
    "artigos_de_natal": "Entretenimento",
    "fashion_roupa_feminina": "Conforto",
    "eletrodomesticos_2": "Essencial",
    "livros_importados": "Entretenimento",
    "bebidas": "Essencial",
    "cine_foto": "Entretenimento",
    "la_cuisine": "Essencial",
    "musica": "Entretenimento",
    "casa_conforto_2": "Conforto",
    "portateis_casa_forno_e_cafe": "Essencial",
    "cds_dvds_musicais": "Entretenimento",
    "dvds_blu_ray": "Entretenimento",
    "flores": "Conforto",
    "artes_e_artesanato": "Entretenimento",
    "fraldas_higiene": "Essencial",
    "fashion_roupa_infanto_juvenil": "Conforto",
    "seguros_e_servicos": "Especializado"
}

# Criação de uma função UDF para mapear as segmentações de funcionalidade principal
def segmentacao_funcionalidade(categoria):
    return segmentacoes_funcionalidade.get(categoria, "Essencial")

# Registro da UDF
segmentacao_funcionalidade_udf = udf(segmentacao_funcionalidade, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacoes_funcionalidade",
    segmentacao_funcionalidade_udf(col("product_category_name"))
)

In [ ]:
# Definição da segmentação por Nível de Interatividade
segmentacoes_interatividade = {
    "beleza_saude": "Média Interatividade",
    "informatica_acessorios": "Alta Interatividade",
    "automotivo": "Média Interatividade",
    "cama_mesa_banho": "Baixa Interatividade",
    "moveis_decoracao": "Baixa Interatividade",
    "esporte_lazer": "Alta Interatividade",
    "perfumaria": "Média Interatividade",
    "utilidades_domesticas": "Média Interatividade",
    "telefonia": "Alta Interatividade",
    "relogios_presentes": "Baixa Interatividade",
    "alimentos_bebidas": "Baixa Interatividade",
    "bebes": "Média Interatividade",
    "papelaria": "Média Interatividade",
    "tablets_impressao_imagem": "Alta Interatividade",
    "brinquedos": "Alta Interatividade",
    "telefonia_fixa": "Alta Interatividade",
    "ferramentas_jardim": "Média Interatividade",
    "fashion_bolsas_e_acessorios": "Baixa Interatividade",
    "eletroportateis": "Média Interatividade",
    "consoles_games": "Alta Interatividade",
    "audio": "Alta Interatividade",
    "fashion_calcados": "Baixa Interatividade",
    "cool_stuff": "Alta Interatividade",
    "malas_acessorios": "Baixa Interatividade",
    "climatizacao": "Média Interatividade",
    "construcao_ferramentas_construcao": "Média Interatividade",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Baixa Interatividade",
    "construcao_ferramentas_jardim": "Média Interatividade",
    "fashion_roupa_masculina": "Baixa Interatividade",
    "pet_shop": "Média Interatividade",
    "moveis_escritorio": "Baixa Interatividade",
    "market_place": "Média Interatividade",
    "eletronicos": "Alta Interatividade",
    "eletrodomesticos": "Média Interatividade",
    "artigos_de_festas": "Média Interatividade",
    "casa_conforto": "Baixa Interatividade",
    "construcao_ferramentas_ferramentas": "Média Interatividade",
    "agro_industria_e_comercio": "Média Interatividade",
    "moveis_colchao_e_estofado": "Baixa Interatividade",
    "livros_tecnicos": "Média Interatividade",
    "casa_construcao": "Média Interatividade",
    "instrumentos_musicais": "Alta Interatividade",
    "moveis_sala": "Baixa Interatividade",
    "construcao_ferramentas_iluminacao": "Média Interatividade",
    "industria_comercio_e_negocios": "Média Interatividade",
    "alimentos": "Baixa Interatividade",
    "artes": "Alta Interatividade",
    "moveis_quarto": "Baixa Interatividade",
    "livros_interesse_geral": "Baixa Interatividade",
    "construcao_ferramentas_seguranca": "Média Interatividade",
    "fashion_underwear_e_moda_praia": "Baixa Interatividade",
    "fashion_esporte": "Alta Interatividade",
    "sinalizacao_e_seguranca": "Média Interatividade",
    "pcs": "Alta Interatividade",
    "artigos_de_natal": "Média Interatividade",
    "fashion_roupa_feminina": "Baixa Interatividade",
    "eletrodomesticos_2": "Média Interatividade",
    "livros_importados": "Baixa Interatividade",
    "bebidas": "Baixa Interatividade",
    "cine_foto": "Alta Interatividade",
    "la_cuisine": "Média Interatividade",
    "musica": "Alta Interatividade",
    "casa_conforto_2": "Baixa Interatividade",
    "portateis_casa_forno_e_cafe": "Média Interatividade",
    "cds_dvds_musicais": "Alta Interatividade",
    "dvds_blu_ray": "Alta Interatividade",
    "flores": "Baixa Interatividade",
    "artes_e_artesanato": "Alta Interatividade",
    "fraldas_higiene": "Média Interatividade",
    "fashion_roupa_infanto_juvenil": "Baixa Interatividade",
    "seguros_e_servicos": "Média Interatividade"
}
# Criação de uma função UDF para mapear as segmentações de funcionalidade principal
def segmentacao_interatividade(categoria):
    return segmentacoes_interatividade.get(categoria, "Média Interatividade")

# Registro da UDF
segmentacao_interatividade_udf = udf(segmentacao_interatividade, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacao_interatividade",
    segmentacao_interatividade_udf(col("product_category_name"))
)


In [ ]:
# Definição da segmentação por Impacto Ambiental
segmentacoes_impacto_ambiental = {
    "beleza_saude": "Médio",
    "informatica_acessorios": "Alto",
    "automotivo": "Alto",
    "cama_mesa_banho": "Baixo",
    "moveis_decoracao": "Médio",
    "esporte_lazer": "Médio",
    "perfumaria": "Médio",
    "utilidades_domesticas": "Médio",
    "telefonia": "Alto",
    "relogios_presentes": "Baixo",
    "alimentos_bebidas": "Baixo",
    "bebes": "Baixo",
    "papelaria": "Baixo",
    "tablets_impressao_imagem": "Alto",
    "brinquedos": "Médio",
    "telefonia_fixa": "Alto",
    "ferramentas_jardim": "Médio",
    "fashion_bolsas_e_acessorios": "Médio",
    "eletroportateis": "Alto",
    "consoles_games": "Alto",
    "audio": "Alto",
    "fashion_calcados": "Médio",
    "cool_stuff": "Alto",
    "malas_acessorios": "Médio",
    "climatizacao": "Alto",
    "construcao_ferramentas_construcao": "Alto",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Baixo",
    "construcao_ferramentas_jardim": "Alto",
    "fashion_roupa_masculina": "Baixo",
    "pet_shop": "Baixo",
    "moveis_escritorio": "Médio",
    "market_place": "Médio",
    "eletronicos": "Alto",
    "eletrodomesticos": "Alto",
    "artigos_de_festas": "Baixo",
    "casa_conforto": "Baixo",
    "construcao_ferramentas_ferramentas": "Alto",
    "agro_industria_e_comercio": "Alto",
    "moveis_colchao_e_estofado": "Médio",
    "livros_tecnicos": "Baixo",
    "casa_construcao": "Alto",
    "instrumentos_musicais": "Médio",
    "moveis_sala": "Médio",
    "construcao_ferramentas_iluminacao": "Alto",
    "industria_comercio_e_negocios": "Alto",
    "alimentos": "Baixo",
    "artes": "Baixo",
    "moveis_quarto": "Médio",
    "livros_interesse_geral": "Baixo",
    "construcao_ferramentas_seguranca": "Alto",
    "fashion_underwear_e_moda_praia": "Baixo",
    "fashion_esporte": "Baixo",
    "sinalizacao_e_seguranca": "Alto",
    "pcs": "Alto",
    "artigos_de_natal": "Baixo",
    "fashion_roupa_feminina": "Baixo",
    "eletrodomesticos_2": "Alto",
    "livros_importados": "Baixo",
    "bebidas": "Baixo",
    "cine_foto": "Alto",
    "la_cuisine": "Baixo",
    "musica": "Baixo",
    "casa_conforto_2": "Baixo",
    "portateis_casa_forno_e_cafe": "Baixo",
    "cds_dvds_musicais": "Baixo",
    "dvds_blu_ray": "Baixo",
    "flores": "Baixo",
    "artes_e_artesanato": "Baixo",
    "fraldas_higiene": "Baixo",
    "fashion_roupa_infanto_juvenil": "Baixo",
    "seguros_e_servicos": "Baixo"
}

# Criação de uma função UDF para mapear as segmentações de funcionalidade principal
def segmentacoe_impacto_ambiental(categoria):
    return segmentacoes_impacto_ambiental.get(categoria, "Média Ambiental")

# Registro da UDF
segmentacoe_impacto_ambiental_udf = udf(segmentacoe_impacto_ambiental, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacoes_impacto_ambiental",
    segmentacoe_impacto_ambiental_udf(col("product_category_name"))
)


In [ ]:
# Definição da segmentação por Interação Humana Necessária
segmentacoes_interacao_humana = {
    "beleza_saude": "Manual",
    "informatica_acessorios": "Assistido",
    "automotivo": "Assistido",
    "cama_mesa_banho": "Manual",
    "moveis_decoracao": "Manual",
    "esporte_lazer": "Manual",
    "perfumaria": "Manual",
    "utilidades_domesticas": "Assistido",
    "telefonia": "Assistido",
    "relogios_presentes": "Manual",
    "alimentos_bebidas": "Manual",
    "bebes": "Manual",
    "papelaria": "Manual",
    "tablets_impressao_imagem": "Assistido",
    "brinquedos": "Manual",
    "telefonia_fixa": "Assistido",
    "ferramentas_jardim": "Manual",
    "fashion_bolsas_e_acessorios": "Manual",
    "eletroportateis": "Assistido",
    "consoles_games": "Assistido",
    "audio": "Assistido",
    "fashion_calcados": "Manual",
    "cool_stuff": "Assistido",
    "malas_acessorios": "Manual",
    "climatizacao": "Assistido",
    "construcao_ferramentas_construcao": "Manual",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "Manual",
    "construcao_ferramentas_jardim": "Manual",
    "fashion_roupa_masculina": "Manual",
    "pet_shop": "Manual",
    "moveis_escritorio": "Manual",
    "market_place": "Assistido",
    "eletronicos": "Assistido",
    "eletrodomesticos": "Assistido",
    "artigos_de_festas": "Manual",
    "casa_conforto": "Manual",
    "construcao_ferramentas_ferramentas": "Manual",
    "agro_industria_e_comercio": "Assistido",
    "moveis_colchao_e_estofado": "Manual",
    "livros_tecnicos": "Manual",
    "casa_construcao": "Manual",
    "instrumentos_musicais": "Manual",
    "moveis_sala": "Manual",
    "construcao_ferramentas_iluminacao": "Manual",
    "industria_comercio_e_negocios": "Assistido",
    "alimentos": "Manual",
    "artes": "Manual",
    "moveis_quarto": "Manual",
    "livros_interesse_geral": "Manual",
    "construcao_ferramentas_seguranca": "Assistido",
    "fashion_underwear_e_moda_praia": "Manual",
    "fashion_esporte": "Manual",
    "sinalizacao_e_seguranca": "Assistido",
    "pcs": "Assistido",
    "artigos_de_natal": "Manual",
    "fashion_roupa_feminina": "Manual",
    "eletrodomesticos_2": "Assistido",
    "livros_importados": "Manual",
    "bebidas": "Manual",
    "cine_foto": "Assistido",
    "la_cuisine": "Manual",
    "musica": "Manual",
    "casa_conforto_2": "Manual",
    "portateis_casa_forno_e_cafe": "Assistido",
    "cds_dvds_musicais": "Manual",
    "dvds_blu_ray": "Manual",
    "flores": "Manual",
    "artes_e_artesanato": "Manual",
    "fraldas_higiene": "Manual",
    "fashion_roupa_infanto_juvenil": "Manual",
    "seguros_e_servicos": "Assistido"
}

# Criação de uma função UDF para mapear as segmentações de funcionalidade principal
def segmentacoe_interacao_humana(categoria):
    return segmentacoes_interacao_humana.get(categoria, "Interação Humana")

# Registro da UDF
segmentacoe_interacao_humana_udf = udf(segmentacoe_interacao_humana, StringType())

# Adição da coluna de segmentação de validade ao DataFrame
products = products.withColumn(
    "segmentacoes_interacao_humana",
    segmentacoe_interacao_humana_udf(col("product_category_name"))
)

In [ ]:
product_category_name_translation.describe()

Out[28]: DataFrame[summary: string, product_category_name: string, product_category_name_english: string]

In [ ]:
# Aplicaremos analise de quartis para eliminar os outliers
merged_data = orders.join(order_payments, on='order_id', how='inner')

# Calcular o valor total por consumidor
total_per_consumer = merged_data.groupBy('customer_id').agg(F.sum('payment_value').alias('total_value'))

# Calcular Q1, Q3 e IQR
Q1 = total_per_consumer.approxQuantile('total_value', [0.10], 0.01)[0]
Q3 = total_per_consumer.approxQuantile('total_value', [0.90], 0.01)[0]
IQR = Q3 - Q1

# Definir limites inferior e superior
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar os consumidores que estão dentro dos limites
valid_customers = total_per_consumer.filter((F.col('total_value') >= lower_bound) & (F.col('total_value') <= upper_bound)).select('customer_id')

# Filtrar as tabelas orders e order_payments para remover os outliers
filtered_orders = orders.join(valid_customers, on='customer_id', how='inner')
filtered_order_payments = order_payments.join(filtered_orders.select('order_id'), on='order_id', how='inner')

# Exibir o número de registros removidos
removed_orders_count = orders.count() - filtered_orders.count()
removed_payments_count = order_payments.count() - filtered_order_payments.count()

print(f"Número de pedidos removidos: {removed_orders_count}")
print(f"Número de pagamentos removidos: {removed_payments_count}")

orders = filtered_orders
order_payments = filtered_order_payments

Número de pedidos removidos: 2595
Número de pagamentos removidos: 2742


In [ ]:
# Unir as tabelas orders e order_items com base no order_id
merged_data = orders.join(order_items, on='order_id', how='inner')

# Agrupar por customer_id e contar o número de produtos distintos (product_id) comprados por cada consumidor
product_count_per_customer = merged_data.groupBy('customer_id').agg(
    F.countDistinct('product_id').alias('distinct_product_count')  # Contar produtos distintos
)

# Filtrar consumidores que compraram mais de um produto
customers_with_multiple_products = product_count_per_customer.filter(
    F.col('distinct_product_count') > 1
)

# Contar quantos consumidores compraram mais de um produto
number_of_customers = customers_with_multiple_products.count()

print(f"Número de consumidores que compraram mais de um product_id: {number_of_customers}")

Número de consumidores que compraram mais de um product_id: 3114


In [ ]:
# Join entre order_items e products para obter todas as informações necessárias
final_order_items = order_items.join(products, on='product_id')

# Aplicar StringIndexer nas colunas de segmentação
categorical_cols = [
    'segmentacao_necessidade', 'segmentacao_sentimento',
    'segmentacao_tipo', 'segmentacao_validade',
    'segmentacao_utilizacao', 'segmentacoes_funcionalidade',
    'segmentacao_interatividade', 'segmentacoes_impacto_ambiental',
    'segmentacoes_interacao_humana'
]

final_order_items_cols = final_order_items.columns
print("Columns in final_order_items after join:", final_order_items_cols)

categorical_cols = [col for col in categorical_cols if col in final_order_items_cols]

indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in categorical_cols]

pipeline = Pipeline(stages=indexers)



Columns in final_order_items after join: ['product_id', 'order_id', 'order_item_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'days_to_shipping_limit', 'shipping_group', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'product_volume', 'segmentacao_necessidade', 'segmentacao_sentimento', 'segmentacao_tipo', 'segmentacao_validade', 'segmentacao_utilizacao', 'segmentacoes_funcionalidade', 'segmentacao_interatividade', 'segmentacoes_impacto_ambiental', 'segmentacoes_interacao_humana']


In [ ]:
# Ajustar e transformar os dados
try:
    final_order_items = pipeline.fit(final_order_items).transform(final_order_items)
except Exception as e:
    print(f"Ocorreu um erro ao aplicar o pipeline: {e}")

# Selecionar as colunas finais
selected_cols = ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
                 'price', 'freight_value', 'days_to_shipping_limit'] + [col + "_index" for col in categorical_cols]

final_order_items = final_order_items.select(selected_cols)

final_order_items.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+----------------------+-----------------------------+----------------------------+----------------------+--------------------------+----------------------------+---------------------------------+--------------------------------+------------------------------------+-----------------------------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|days_to_shipping_limit|segmentacao_necessidade_index|segmentacao_sentimento_index|segmentacao_tipo_index|segmentacao_validade_index|segmentacao_utilizacao_index|segmentacoes_funcionalidade_index|segmentacao_interatividade_index|segmentacoes_impacto_ambiental_index|segmentacoes_interacao_humana_index|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+----------------------+----------------

In [ ]:
merged_data.describe()

Out[33]: DataFrame[summary: string, order_id: string, customer_id: string, order_status: string, days_to_approval: string, days_to_carrier: string, days_to_delivery: string, days_estimated_vs_actual_delivery: string, approval_group: string, carrier_group: string, delivery_group: string, estimated_vs_actual_group: string, order_item_id: string, product_id: string, seller_id: string, price: string, freight_value: string, days_to_shipping_limit: string, shipping_group: string]

In [ ]:

# Definir os indexers para os DataFrames corretos
indexers_orders = [
    StringIndexer(inputCol="order_status", outputCol="order_status_index"),
]

indexers_sellers = [
    StringIndexer(inputCol="seller_city", outputCol="seller_city_index"),
    StringIndexer(inputCol="seller_state", outputCol="seller_state_index"),
]

indexers_customers = [
    StringIndexer(inputCol="customer_state", outputCol="customer_state_index"),
]

indexers_payments = [
    StringIndexer(inputCol="payment_type", outputCol="payment_type_index"),
]

# Aplicar os indexers para orders
for indexer in indexers_orders:
    output_col = indexer.getOutputCol()
    if output_col in orders.columns:
        orders = orders.drop(output_col)
    orders = indexer.fit(orders).transform(orders)

# Aplicar os indexers para sellers
for indexer in indexers_sellers:
    output_col = indexer.getOutputCol()
    if output_col in sellers.columns:
        sellers = sellers.drop(output_col)
    sellers = indexer.fit(sellers).transform(sellers)

# Aplicar os indexers para customers
for indexer in indexers_customers:
    output_col = indexer.getOutputCol()
    if output_col in customers.columns:
        customers = customers.drop(output_col)
    customers = indexer.fit(customers).transform(customers)

# Aplicar os indexers para order_payments
for indexer in indexers_payments:
    output_col = indexer.getOutputCol()
    if output_col in order_payments.columns:
        order_payments = order_payments.drop(output_col)
    order_payments = indexer.fit(order_payments).transform(order_payments)

# Selecionar mês da order_purchase_timestamp
orders = orders.withColumn('order_purchase_month', month(col('order_purchase_timestamp')))

# Join entre order_items e products para obter as categorias dos produtos
order_items = order_items.join(products, on='product_id', how='inner')

# Transformar colunas categóricas de produtos
categorical_cols_products = [
    'segmentacao_necessidade', 'segmentacao_sentimento',
    'segmentacao_tipo', 'segmentacao_validade',
    'segmentacao_utilizacao', 'segmentacoes_funcionalidade',
    'segmentacao_interatividade', 'segmentacoes_impacto_ambiental',
    'segmentacoes_interacao_humana'
]

indexers_products = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in categorical_cols_products]
pipeline_products = Pipeline(stages=indexers_products)
products = pipeline_products.fit(products).transform(products)

# Join entre order_items e products com colunas transformadas
order_items = order_items.join(products, on='product_id', how='inner')

# Consolidar todas as features selecionadas com inner joins
df = order_reviews.select('order_id', col('review_score').cast('int').alias('target')) \
    .join(orders.select('order_id', 'customer_id', 'order_status_index', 'order_purchase_month',
                        'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date',
                        'order_estimated_delivery_date', 'days_to_approval', 'days_to_carrier',
                        'days_to_delivery', 'days_estimated_vs_actual_delivery'),
          on='order_id', how='inner') \
    .join(order_items.select('order_id', 'product_id', 'price', 'freight_value', 'seller_id',
                             *[col + "_index" for col in categorical_cols_products]), on='order_id', how='inner') \
    .join(sellers.select('seller_id', 'seller_city_index', 'seller_state_index'), on='seller_id', how='inner') \
    .join(customers.select('customer_id', 'customer_state_index'), on='customer_id', how='inner') \
    .join(order_payments.select('order_id', 'payment_sequential', 'payment_type_index',
                                 'payment_installments', 'payment_value'), on='order_id', how='inner')

# Selecionar apenas as colunas necessárias para o modelo, incluindo novas colunas de duração
df = df.select('target', 'order_status_index', 'order_purchase_month',
               'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date',
               'order_estimated_delivery_date', 'days_to_approval', 'days_to_carrier',
               'days_to_delivery', 'days_estimated_vs_actual_delivery',
               'price', 'freight_value',
               'segmentacao_necessidade_index',
               'segmentacao_sentimento_index',
               'segmentacao_tipo_index',
               'segmentacao_validade_index',
               'segmentacao_utilizacao_index',
               'segmentacoes_funcionalidade_index',
               'segmentacao_interatividade_index',
               'segmentacoes_impacto_ambiental_index',
               'segmentacoes_interacao_humana_index',
               'seller_city_index',
               'seller_state_index',
               'customer_state_index',
               'payment_sequential',
               'payment_type_index',
               'payment_installments',
               'payment_value')

# Filtrar apenas linhas onde a coluna target (review_score) não é nula
df = df.filter(df.target.isNotNull())

# Tratar valores nulos (se necessário)
df = df.na.drop()  # Remove linhas com qualquer valor nulo (se ainda houver)

# Visualizar o DataFrame consolidado
#df.show()

# Contar o número de linhas no DataFrame filtrado
#num_linhas_df = df.count()
#print(f"Número de linhas no DataFrame filtrado: {num_linhas_df}")


In [ ]:
# Converter colunas para os tipos adequados
df = df.withColumn("order_status_index", col("order_status_index").cast("double"))
df = df.withColumn("order_purchase_month", col("order_purchase_month").cast("double"))
df = df.withColumn("days_to_approval", col("days_to_approval").cast("double"))
df = df.withColumn("days_to_carrier", col("days_to_carrier").cast("double"))
df = df.withColumn("days_to_delivery", col("days_to_delivery").cast("double"))
df = df.withColumn("days_estimated_vs_actual_delivery", col("days_estimated_vs_actual_delivery").cast("double"))
df = df.withColumn("price", col("price").cast("double"))
df = df.withColumn("freight_value", col("freight_value").cast("double"))
df = df.withColumn("segmentacao_necessidade_index", col("segmentacao_necessidade_index").cast("double"))
df = df.withColumn("seller_city_index", col("seller_city_index").cast("double"))
df = df.withColumn("seller_state_index", col("seller_state_index").cast("double"))
df = df.withColumn("customer_state_index", col("customer_state_index").cast("double"))
df = df.withColumn("payment_sequential", col("payment_sequential").cast("double"))
df = df.withColumn("payment_type_index", col("payment_type_index").cast("double"))
df = df.withColumn("payment_installments", col("payment_installments").cast("double"))
df = df.withColumn("payment_value", col("payment_value").cast("double"))
df = df.withColumn('seller_city_index', col('seller_city_index').cast("double"))
df = df.withColumn('seller_state_index', col('seller_state_index').cast("double"))
df = df.withColumn('customer_state_index', col('customer_state_index').cast("double"))

#### Selecionar as colunas que queremos normalizar
numeric_cols = ['order_status_index', 'order_purchase_month', 'days_to_approval', 'days_to_carrier',
                'days_to_delivery', 'days_estimated_vs_actual_delivery', 'price', 'freight_value',
                'segmentacao_necessidade_index', 'segmentacao_sentimento_index', 'segmentacao_tipo_index',
                'segmentacao_validade_index', 'segmentacao_utilizacao_index', 'segmentacoes_funcionalidade_index',
                'segmentacao_interatividade_index', 'segmentacoes_impacto_ambiental_index','seller_state_index', 'seller_state_index','segmentacoes_interacao_humana_index', 'payment_sequential', 'payment_type_index', 'payment_installments',
                'payment_value']

assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")

df_assembled = assembler.transform(df)

scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)

scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

# Visualizar o DataFrame final
df_scaled.select("target", "scaled_features").show()


+------+--------------------+
|target|     scaled_features|
+------+--------------------+
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     4|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     4|[-0.0079415832780...|
|     4|[-0.0079415832780...|
|     4|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     4|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
|     5|[-0.0079415832780...|
+------+--------------------+
only showing top 20 rows



In [ ]:
train_df, test_df = df_scaled.randomSplit([0.8, 0.2], seed=42)

label_col = 'target'

modelos = {
    'Regressão Linear': LinearRegression(featuresCol='scaled_features', labelCol=label_col),
    'Árvore de Decisão': DecisionTreeRegressor(featuresCol='scaled_features', labelCol=label_col),
    'Random Forest': RandomForestRegressor(featuresCol='scaled_features', labelCol=label_col),
    'Gradient Boosting': GBTRegressor(featuresCol='scaled_features', labelCol=label_col)
}

# Avaliador
avaliador = RegressionEvaluator(labelCol=label_col, predictionCol='prediction', metricName='rmse')

# Treinamento e avaliação dos modelos
resultados = []

for nome, modelo in modelos.items():
    # Treinar o modelo
    modelo_treinado = modelo.fit(train_df)

    # Fazer previsões no conjunto de teste
    previsoes = modelo_treinado.transform(test_df)

    # Avaliar o modelo
    rmse = avaliador.evaluate(previsoes)

    # Armazenar os resultados
    resultados.append((nome, rmse))
    print(f'{nome} - RMSE: {rmse}')

### Converter os resultados para DataFrame Pandas para visualização
resultados_df = pd.DataFrame(resultados, columns=['Modelo', 'RMSE'])
print(resultados_df)

# Escolher o melhor modelo baseado no RMSE
melhor_modelo_nome = resultados_df.loc[resultados_df['RMSE'].idxmin(), 'Modelo']
print(f'Melhor modelo: {melhor_modelo_nome}')

# Treinar o melhor modelo com todo o dataset
melhor_modelo = modelos[melhor_modelo_nome].fit(df_scaled)

# Fazer previsões com o melhor modelo (por exemplo, no conjunto de teste)
previsoes_finais = melhor_modelo.transform(test_df)
previsoes_finais.select('scaled_features', 'target', 'prediction').show(5)




Regressão Linear - RMSE: 1.241404849786044
Árvore de Decisão - RMSE: 1.204637493272893
Random Forest - RMSE: 1.2067568629432472
Gradient Boosting - RMSE: 1.181294895136165
              Modelo      RMSE
0   Regressão Linear  1.241405
1  Árvore de Decisão  1.204637
2      Random Forest  1.206757
3  Gradient Boosting  1.181295
Melhor modelo: Gradient Boosting
+--------------------+------+-----------------+
|     scaled_features|target|       prediction|
+--------------------+------+-----------------+
|[-0.0079415832780...|     1|4.257300305151925|
|[-0.0079415832780...|     1|1.508959670983282|
|[-0.0079415832780...|     1| 4.23967317799092|
|[-0.0079415832780...|     1|4.071637025196427|
|[-0.0079415832780...|     1|3.650379323833151|
+--------------------+------+-----------------+
only showing top 5 rows



In [ ]:
importances = melhor_modelo.featureImportances

feature_importances = [(feature, importance) for feature, importance in zip(numeric_cols, importances)]

sorted_feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)

for feature, importance in sorted_feature_importances:
    print(f'Feature: {feature}, Importance: {importance}')


Feature: days_estimated_vs_actual_delivery, Importance: 0.2744845374197612
Feature: price, Importance: 0.19375324514687392
Feature: payment_value, Importance: 0.1519064343921461
Feature: freight_value, Importance: 0.06830783388381916
Feature: days_to_delivery, Importance: 0.06615620718443968
Feature: order_purchase_month, Importance: 0.042189827595033
Feature: days_to_carrier, Importance: 0.04159337507187227
Feature: payment_sequential, Importance: 0.021395801329828667
Feature: segmentacao_necessidade_index, Importance: 0.018520734273497135
Feature: segmentacao_tipo_index, Importance: 0.016822092180999525
Feature: seller_state_index, Importance: 0.015761441548751403
Feature: segmentacao_utilizacao_index, Importance: 0.015128509765602394
Feature: payment_installments, Importance: 0.013602826688264642
Feature: days_to_approval, Importance: 0.013252429614374126
Feature: segmentacoes_impacto_ambiental_index, Importance: 0.013175544846275752
Feature: segmentacao_interatividade_index, Import

In [ ]:
# Verificar os índices únicos na coluna 'customer_state_index'
estado_indices = test_df_modificado.select('customer_state_index').distinct().collect()

# Exibir os índices únicos
print("Índices únicos na coluna 'customer_state_index':")
for row in estado_indices:
    print(row['customer_state_index'])


Índices únicos na coluna 'customer_state_index':
8.0
0.0
7.0
18.0
1.0
25.0
4.0
23.0
11.0
21.0
14.0
22.0
19.0
3.0
2.0
17.0
10.0
13.0
6.0
20.0
5.0
15.0
9.0
16.0
12.0
24.0
26.0


In [ ]:
# Selecionar e exibir os estados únicos na coluna 'customer_state' no DataFrame 'customers'
customers.select('customer_state').distinct().show()


+--------------+
|customer_state|
+--------------+
|            SC|
|            RO|
|            AM|
|            GO|
|            MT|
|            SP|
|            ES|
|            RS|
|            MS|
|            AL|
|            MG|
|            PA|
|            BA|
|            PE|
|            CE|
|            RN|
|            RJ|
|            MA|
|            DF|
|            PR|
+--------------+
only showing top 20 rows



In [ ]:
estado_indices

Out[61]: [Row(customer_state_index=8.0),
 Row(customer_state_index=0.0),
 Row(customer_state_index=7.0),
 Row(customer_state_index=18.0),
 Row(customer_state_index=1.0),
 Row(customer_state_index=4.0),
 Row(customer_state_index=23.0),
 Row(customer_state_index=11.0),
 Row(customer_state_index=21.0),
 Row(customer_state_index=14.0),
 Row(customer_state_index=22.0),
 Row(customer_state_index=3.0),
 Row(customer_state_index=19.0),
 Row(customer_state_index=2.0),
 Row(customer_state_index=17.0),
 Row(customer_state_index=10.0),
 Row(customer_state_index=13.0),
 Row(customer_state_index=6.0),
 Row(customer_state_index=20.0),
 Row(customer_state_index=5.0),
 Row(customer_state_index=15.0),
 Row(customer_state_index=24.0),
 Row(customer_state_index=9.0),
 Row(customer_state_index=26.0),
 Row(customer_state_index=16.0),
 Row(customer_state_index=12.0),
 Row(customer_state_index=25.0)]

In [ ]:
# Calcular a média dos review_scores antes das modificações
media_antes = test_df.select(avg('target')).first()[0]
print(f'Média dos review_scores antes das modificações: {media_antes}')

# Definindo os índices correspondentes para SP e RJ
indice_sp = 25
indice_rj = 16

# Filtrar dados para clientes de São Paulo (SP) e Rio de Janeiro (RJ)
clientes_sp_rj = test_df_modificado.filter(col('customer_state_index').isin([indice_sp, indice_rj]))

# Filtrar clientes com review_scores baixos (indicando insatisfação)
clientes_insatisfeitos = clientes_sp_rj.filter(col('target') < 3)
clientes_insatisfeitos.show()

# Analisar a quantidade de clientes insatisfeitos em cada estado
clientes_insatisfeitos.groupBy('customer_state_index').count().show()

# Adicionar desconto especial para clientes insatisfeitos de SP e RJ
clientes_insatisfeitos_modificado = clientes_insatisfeitos.withColumn('price', col('price') * 0.80)  # 20% de desconto

# Oferecer garantia estendida gratuita para aumentar a confiança do cliente
#clientes_insatisfeitos_modificado = clientes_insatisfeitos_modificado.withColumn('extended_warranty', lit(1))

# Adicionar mensagens personalizadas para campanhas em SP e RJ
#clientes_insatisfeitos_modificado = clientes_insatisfeitos_modificado.withColumn('campaign_message',
#    when(col('customer_state_index') == indice_sp, lit("Ofertas exclusivas em São Paulo! Aproveite 20% de desconto e garantia estendida gratuita."))
#    .when(col('customer_state_index') == indice_rj, lit("Ofertas exclusivas no Rio de Janeiro! Aproveite 20% de desconto e garantia estendida gratuita."))
#    .otherwise(lit("Ofertas exclusivas! Aproveite 20% de desconto e garantia estendida gratuita.")))

# Implementar uma coluna de feedback para monitorar a eficácia das campanhas
#clientes_modificado = clientes_insatisfeitos_modificado.withColumn('feedback_provided', lit(0))

# Visualizar os dados finais para campanhas direcionadas
#clientes_modificado.select('customer_state_index', 'price', 'extended_warranty', 'campaign_message', 'feedback_provided').show()





# Reduzir o valor do pagamento em 5% para proporcionar economia ao cliente
test_df_modificado = test_df.withColumn('payment_value', col('payment_value') * 0.85)

# Ajustar o preço baseado na sazonalidade (manter preço inalterado nos meses 5 e 6)
test_df_modificado = test_df_modificado.withColumn('price', when(col('order_purchase_month').isin([5, 6]), col('price') * 1.0).otherwise(col('price')))

# Adicionar informação de rastreamento disponível para aumentar a transparência
#test_df_modificado = test_df_modificado.withColumn('tracking_info_available', lit(1))

# Incluir clientes em estados específicos no programa de fidelidade
#test_df_modificado = test_df_modificado.withColumn('loyalty_program', when(col('customer_state_index').isin([1, 2]), 1).otherwise(0))

# Oferecer garantia estendida para aumentar a confiança do cliente
#test_df_modificado = test_df_modificado.withColumn('extended_warranty', lit(1))

# Ajustar o número de parcelas com base nas observações (reduzir parcelas se maior que 6)
test_df_modificado = test_df_modificado.withColumn('payment_installments', when(col('payment_installments') > 6, col('payment_installments') - 3).otherwise(col('payment_installments')))

# Adicionar ajustes baseados nas segmentações

# Aumentar o preço em 10% se segmentação de necessidade for 1
test_df_modificado = test_df_modificado.withColumn('price', when(col('segmentacao_necessidade_index') == 1, col('price') * 1.10).otherwise(col('price')))

# Reduzir o preço em 10% se segmentação de sentimento for 2
test_df_modificado = test_df_modificado.withColumn('price', when(col('segmentacao_sentimento_index') == 2, col('price') * 0.90).otherwise(col('price')))

# Aumentar o preço em 15% se segmentação de tipo for 3
test_df_modificado = test_df_modificado.withColumn('price', when(col('segmentacao_tipo_index') == 3, col('price') * 1.15).otherwise(col('price')))

# Reduzir o preço em 5% se segmentação de validade for 4
test_df_modificado = test_df_modificado.withColumn('price', when(col('segmentacao_validade_index') == 4, col('price') * 0.95).otherwise(col('price')))

# Fazer previsões com o modelo ajustado
previsoes_modificado = melhor_modelo.transform(test_df_modificado)

# Calcular a média dos review_scores depois das modificações
media_depois = previsoes_modificado.select(avg('prediction')).first()[0]
print(f'Média dos review_scores depois das modificações: {media_depois}')


+------+------------------+--------------------+-------------------+----------------------------+-----------------------------+-----------------------------+----------------+---------------+----------------+---------------------------------+-----------------+-------------+-----------------------------+----------------------------+----------------------+--------------------------+----------------------------+---------------------------------+--------------------------------+------------------------------------+-----------------------------------+-----------------+------------------+--------------------+------------------+------------------+--------------------+------------------+--------------------+--------------------+-----------------------+---------------+-----------------+
|target|order_status_index|order_purchase_month|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|days_to_approval|days_to_carrier|days_to_delivery|days_

In [ ]:
# Mostrar o esquema do DataFrame test_df_modificado
test_df_modificado.printSchema()



root
 |-- target: integer (nullable = true)
 |-- order_status_index: double (nullable = false)
 |-- order_purchase_month: double (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- days_to_approval: double (nullable = true)
 |-- days_to_carrier: double (nullable = true)
 |-- days_to_delivery: double (nullable = true)
 |-- days_estimated_vs_actual_delivery: double (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- segmentacao_necessidade_index: double (nullable = false)
 |-- segmentacao_sentimento_index: double (nullable = false)
 |-- segmentacao_tipo_index: double (nullable = false)
 |-- segmentacao_validade_index: double (nullable = false)
 |-- segmentacao_utilizacao_index: double (nullable = false)
 |-- segmentacoes_func

In [ ]:
# Ajustar o StringIndexer na coluna 'customer_state'
indexer = StringIndexer(inputCol="customer_state", outputCol="customer_state_index")
model = indexer.fit(test_df_modificado)

# Transformar o DataFrame para adicionar a coluna 'customer_state_index'
df_original_indexed = model.transform(test_df_modificado)

# Exibir os mapeamentos de estados para índices
df_original_indexed.show()

# Criar o mapeamento de estados para índices
mapping = dict(zip(model.labels, range(len(model.labels))))
print(mapping)

# Consultar os índices correspondentes para BA e RJ
indice_ba = mapping.get('BA')
indice_rj = mapping.get('RJ')
print(f'Índice para BA: {indice_ba}')
print(f'Índice para RJ: {indice_rj}')


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-361214760284451>:3
      1 # Ajustar o StringIndexer na coluna 'customer_state'
      2 indexer = StringIndexer(inputCol="customer_state", outputCol="customer_state_index")
----> 3 model = indexer.fit(test_df_modificado)
      5 # Transformar o DataFrame para adicionar a coluna 'customer_state_index'
      6 df_original_indexed = model.transform(test_df_modificado)

File /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py:30, in _create_patch_function.<locals>.patched_method(self, *args, **kwargs)
     28 call_succeeded = False
     29 try:
---> 30     result = original_method(self, *args, **kwargs)
     31     call_succeeded = True
     32     return result

File /databricks/spark/python/pyspark/ml/base.py:205, in Estimator.fit(self, dataset, params)
    203         return self.copy(params)._fit(data

In [ ]:
from pyspark.sql.functions import lit, when

# Calcular a média dos review_scores antes das modificações
media_antes = test_df.select(avg('target')).first()[0]aaaaaaaaaaaaaaaaaaaaaaaaaa
print(f'Média dos review_scores antes das modificações: {media_antes}')

# Aplicar novas modificações sugeridas
test_df_modificado = test_df_modificado.withColumn('payment_installments', col('payment_installments') + 2)
test_df_modificado = test_df_modificado.withColumn('payment_value', col('payment_value') * 0.95)
test_df_modificado = test_df_modificado.withColumn('price', when(col('order_purchase_month').isin([5, 6]), col('price') * 1.0).otherwise(col('price')))
test_df_modificado = test_df_modificado.withColumn('tracking_info_available', lit(1))
test_df_modificado = test_df_modificado.withColumn('loyalty_program', when(col('customer_state_index') < 4, 1).otherwise(0))
test_df_modificado = test_df_modificado.withColumn('extended_warranty', lit(1))
test_df_modificado = test_df_modificado.withColumn('easy_return', lit(1))

# Fazer previsões com o modelo ajustado
previsoes_modificado = melhor_modelo.transform(test_df_modificado)

# Calcular a média dos review_scores depois das modificações
media_depois = previsoes_modificado.select(avg('prediction')).first()[0]
print(f'Média dos review_scores depois das modificações: {media_depois}')


  File <command-1752482484161537>:4
    media_antes = test_df.select(avg('target')).first()[0]aaaaaaaaaaaaaaaaaaaaaaaaaa
                                                          ^
SyntaxError: invalid syntax


In [ ]:
from pyspark.sql.functions import col, lit, when, avg

# Calcular a média dos review_scores antes das modificações
media_antes = test_df.select(avg('target')).first()[0]aaaaaaaaaaaaaa
print(f'Média dos review_scores antes das modificações: {media_antes}')

# Aplicar novas modificações sugeridas
test_df_modificado = test_df.withColumn('payment_value', col('payment_value') * 0.95)
test_df_modificado = test_df_modificado.withColumn('price', when(col('order_purchase_month').isin([5, 6]), col('price') * 1.0).otherwise(col('price')))
test_df_modificado = test_df_modificado.withColumn('tracking_info_available', lit(1))
test_df_modificado = test_df_modificado.withColumn('loyalty_program', when(col('customer_state_index').isin([1, 2]), 1).otherwise(0))
test_df_modificado = test_df_modificado.withColumn('extended_warranty', lit(1))
# test_df_modificado = test_df_modificado.withColumn('easy_return', lit(1))

# Ajustar o número de parcelas com base nas observações
test_df_modificado = test_df_modificado.withColumn('payment_installments', when(col('payment_installments') > 6, col('payment_installments') - 2).otherwise(col('payment_installments')))

# Adicionar ajustes baseados nas segmentações
test_df_modificado = test_df_modificado.withColumn('price', when(col('segmentacao_necessidade_index') == 1, col('price') * 1.10).otherwise(col('price')))
test_df_modificado = test_df_modificado.withColumn('price', when(col('segmentacao_sentimento_index') == 2, col('price') * 0.90).otherwise(col('price')))
test_df_modificado = test_df_modificado.withColumn('price', when(col('segmentacao_tipo_index') == 3, col('price') * 1.15).otherwise(col('price')))
test_df_modificado = test_df_modificado.withColumn('price', when(col('segmentacao_validade_index') == 4, col('price') * 0.95).otherwise(col('price')))

# Fazer previsões com o modelo ajustado
previsoes_modificado = melhor_modelo.transform(test_df_modificado)

# Calcular a média dos review_scores depois das modificações
media_depois = previsoes_modificado.select(avg('prediction')).first()[0]
print(f'Média dos review_scores depois das modificações: {media_depois}')
